<h1 style="text-align: center;">MIS 542 &ndash; Advanced Business Analytics</h1>
<h1 style="text-align: center;">Second Business Case - Taxi Cancellation</h1>

<h4> Team 7 Members - Section 1:
<h4> Nada Alomari - 202111090 | Saja Alqahtani - 202111070 | Reda Alhamza - 201365390

In [1]:
import pandas as pd

taxi_df = pd.read_csv('/Users/nada/Downloads/Taxi-cancellation.csv')

In [2]:
taxi_df.head()

,row#,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
0,1,17712,12,NaN,2,1021.0,1323.0,NaN,NaN,1/1/13 22:33,NaN,0,0,1/1/13 8:01,13.028530,77.54625,12.869805,77.653211,0
1,2,17037,12,NaN,2,455.0,1330.0,NaN,NaN,1/1/13 12:43,NaN,0,0,1/1/13 9:59,12.999874,77.67812,12.953434,77.706510,0
2,3,761,12,NaN,2,814.0,393.0,NaN,NaN,1/2/13 0:28,1/3/13 0:00,1,0,1/1/13 12:14,12.908993,77.68890,13.199560,77.706880,0
3,4,868,12,NaN,2,297.0,212.0,NaN,NaN,1/1/13 13:12,NaN,0,0,1/1/13 12:42,12.997890,77.61488,12.994740,77.607970,0
4,5,21716,28,NaN,2,1237.0,330.0,NaN,NaN,1/1/13 16:33,NaN,0,0,1/1/13 15:07,12.926450,77.61206,12.858833,77.589127,0


In [3]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   row#                 10000 non-null  int64  
 1   user_id              10000 non-null  int64  
 2   vehicle_model_id     10000 non-null  int64  
 3   package_id           1752 non-null   float64
 4   travel_type_id       10000 non-null  int64  
 5   from_area_id         9985 non-null   float64
 6   to_area_id           7909 non-null   float64
 7   from_city_id         3706 non-null   float64
 8   to_city_id           339 non-null    float64
 9   from_date            10000 non-null  object 
 10  to_date              5822 non-null   object 
 11  online_booking       10000 non-null  int64  
 12  mobile_site_booking  10000 non-null  int64  
 13  booking_created      10000 non-null  object 
 14  from_lat             9985 non-null   float64
 15  from_long            9985 non-null   

<h4> 1. Set “0” to all missing values in the table

In [4]:
# 1. Set “0” to all missing values in the table
import numpy as np  
taxi_df = taxi_df.replace(np.nan, 0)

<h4> 2. Use the following variables as predictors: [‘travel_type_id', 'within_city',
'online_booking', 'mobile_site_booking', 'booking_created_Hour', 'trip_length'], and
[‘Car_Cancellation’] as an outcome variable.
a. 'within_city' is a new binary variable that you will create and assign “1” when
['from_city_id'] == taxi_df['to_city_id'].



In [5]:

predictors = ['travel_type_id', 'within_city','online_booking', 
             'mobile_site_booking', 'booking_created_Hour', 'trip_length']
outcome = 'Car_Cancellation'

<h4> a.'within_city' is a new binary variable that you will create and assign “1” when ['from_city_id'] == taxi_df['to_city_id'].



In [6]:

for i in range(0, taxi_df.shape[0]-1):
    taxi_df['within_city'] = 1 if taxi_df['to_city_id'][i] == taxi_df['from_city_id'][i] else 0
    
for i in range(0, taxi_df.shape[0]-1):
    taxi_df['within_city'][i] = 1 if taxi_df['to_city_id'][i] == taxi_df['from_city_id'][i] else 0

<ipython-input-6-c1f4433c343f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxi_df['within_city'][i] = 1 if taxi_df['to_city_id'][i] == taxi_df['from_city_id'][i] else 0


In [7]:
set(taxi_df["within_city"])

{0, 1}

In [8]:
taxi_df["within_city"].value_counts()

1    6241
0    3759
Name: within_city, dtype: int64

<h4> b. 'booking_created_Hour' is derived from the existing variable [‘booking_created’]

In [9]:
taxi_df['booking_created'].head()

0     1/1/13 8:01
1     1/1/13 9:59
2    1/1/13 12:14
3    1/1/13 12:42
4    1/1/13 15:07
Name: booking_created, dtype: object

In [10]:
taxi_df['booking_created'] = taxi_df['booking_created'].astype('datetime64[ns]')

In [11]:
taxi_df['booking_created_Hour'] = taxi_df.booking_created.dt.hour

In [12]:
taxi_df['booking_created_Hour'].head()

0     8
1     9
2    12
3    12
4    15
Name: booking_created_Hour, dtype: int64

<h4> c. 'trip_length' is basically the distance calculated from the GPS data; use the following four variables [‘from_lat’, ‘from_long’, ‘to_lat’ ‘to_long’] to derive the distance and name it ‘trip_length’.

In [13]:
# Python 3 program to calculate Distance Between Two Points on Earth
def distance(lat1, lat2, lon1, lon2):
     
    # radians which converts from degrees to radians.
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
 
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

#References:
#Latitude/Longitude Distance Calculator. Latitude/Longitude Distance Calculator - Meridian Outpost. (n.d.). Retrieved March 25, 2022, from https://www.meridianoutpost.com/resources/etools/calculators/calculator-latitude-longitude-distance.php 
#Program for distance between two points on Earth. GeeksforGeeks. (2021, April 9). Retrieved March 25, 2022, from https://www.geeksforgeeks.org/program-distance-two-points-earth/ 

In [14]:
taxi_df['trip_length'] = distance(taxi_df['from_lat'], taxi_df['to_lat'], taxi_df['from_long'], taxi_df['to_long'])
                                                                                  

NumExpr defaulting to 8 threads.


In [36]:
taxi_df.head()

,row#,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,...,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation,within_city,booking_created_Hour,trip_length
0,1,17712,12,0.0,2,1021.0,1323.0,0.0,0.0,1/1/13 22:33,...,0,2013-01-01 08:01:00,13.028530,77.54625,12.869805,77.653211,0,1,8,21.115265
1,2,17037,12,0.0,2,455.0,1330.0,0.0,0.0,1/1/13 12:43,...,0,2013-01-01 09:59:00,12.999874,77.67812,12.953434,77.706510,0,1,9,6.010725
2,3,761,12,0.0,2,814.0,393.0,0.0,0.0,1/2/13 0:28,...,0,2013-01-01 12:14:00,12.908993,77.68890,13.199560,77.706880,0,1,12,32.368224
3,4,868,12,0.0,2,297.0,212.0,0.0,0.0,1/1/13 13:12,...,0,2013-01-01 12:42:00,12.997890,77.61488,12.994740,77.607970,0,1,12,0.826559
4,5,21716,28,0.0,2,1237.0,330.0,0.0,0.0,1/1/13 16:33,...,0,2013-01-01 15:07:00,12.926450,77.61206,12.858833,77.589127,0,1,15,7.918920


<h4> Detecting & Omitting Outliers 

In [15]:
# Any trip length value that's more than 100 km is considered as an outlier as below.
taxi_df[taxi_df['trip_length'] > 100]

,row#,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,...,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation,within_city,booking_created_Hour,trip_length
8,9,22201,12,1.0,3,1323.0,0.0,0.0,0.0,1/1/13 19:55,...,0,2013-01-01 17:25:00,12.869805,77.653211,0.0,0.0,0,1,17,8669.655202
12,13,15846,12,2.0,3,142.0,0.0,0.0,0.0,1/2/13 3:07,...,0,2013-01-01 19:37:00,12.912810,77.609230,0.0,0.0,0,1,19,8665.127374
23,24,15529,12,2.0,3,217.0,0.0,0.0,0.0,1/2/13 14:24,...,0,2013-01-02 09:04:00,13.001100,77.623500,0.0,0.0,0,1,9,8667.154585
25,26,22229,12,2.0,3,1053.0,0.0,0.0,0.0,1/2/13 11:31,...,0,2013-01-02 10:29:00,12.912799,77.589784,0.0,0.0,0,1,10,8663.022238
29,30,18777,12,1.0,3,293.0,0.0,0.0,0.0,1/2/13 17:02,...,0,2013-01-02 12:15:00,12.849482,77.663187,0.0,0.0,0,1,12,8670.625465
47,48,41448,12,1.0,3,767.0,0.0,0.0,0.0,1/5/13 14:24,...,0,2013-01-03 10:32:00,12.912200,77.510430,0.0,0.0,0,1,10,8654.428773
60,61,22236,24,2.0,3,1191.0,0.0,0.0,0.0,1/4/13 12:28,...,0,2013-01-04 11:24:00,13.030170,77.566840,0.0,0.0,0,1,11,8661.183592
61,62,22318,65,0.0,1,293.0,0.0,15.0,32.0,1/5/13 7:55,...,0,2013-01-04 12:43:00,12.849482,77.663187,0.0,0.0,0,0,12,8670.625465
62,63,19126,12,2.0,3,145.0,0.0,0.0,0.0,1/4/13 15:50,...,0,2013-01-04 14:23:00,12.990670,77.655870,0.0,0.0,0,1,14,8670.600437
73,74,22352,28,2.0,3,115.0,0.0,0.0,0.0,1/6/13 1:26,...,0,2013-01-04 21:02:00,12.993190,77.544720,0.0,0.0,0,1,21,8658.586015


In [16]:
taxi_df = taxi_df[taxi_df['trip_length'] <= 100]

In [17]:
taxi_df.shape

(7924, 22)

<h4> 3. Split the data into two sets; training (60%) and test (40%)



In [18]:
x= taxi_df[predictors]
y=taxi_df[outcome]
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.4, random_state = 1)

<h4> 4. Apply three ML techniques (Decision Trees, Random Forest, and Neural Net) for this
classification problem.

<h5> Decision Trees

In [19]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
from dmba import plotDecisionTree, classificationSummary, regressionSummary

no display found. Using non-interactive Agg backend


In [20]:
fullClassTree = DecisionTreeClassifier()
fullClassTree.fit(train_x, train_y)
classificationSummary(train_y, fullClassTree.predict(train_x))
classificationSummary(test_y, fullClassTree.predict(test_x))

Confusion Matrix (Accuracy 0.9933)

       Prediction
Actual    0    1
     0 4350   10
     1   22  372
Confusion Matrix (Accuracy 0.8662)

       Prediction
Actual    0    1
     0 2685  233
     1  191   61


In [21]:
DT_table = pd.crosstab(test_y, fullClassTree.predict(test_x)) 

<h5> Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [23]:
rf = RandomForestClassifier(n_estimators=500, random_state=1)
rf.fit(train_x, train_y)
classificationSummary(test_y, rf.predict(test_x))
importances = rf.feature_importances_

Confusion Matrix (Accuracy 0.8893)

       Prediction
Actual    0    1
     0 2778  140
     1  211   41


In [24]:
RF_table = pd.crosstab(test_y, rf.predict(test_x)) 

In [25]:
df = pd.DataFrame({'feature':train_x.columns, 'importance': importances})
df = df.sort_values('importance')
print(df)

                feature  importance
0        travel_type_id    0.002154
3   mobile_site_booking    0.010230
1           within_city    0.018039
2        online_booking    0.033976
4  booking_created_Hour    0.184721
5           trip_length    0.750880


<h5> Neural Net

In [26]:
from pathlib import Path
import pandas as pd
from sklearn.neural_network import MLPClassifier
from dmba import classificationSummary

In [27]:
classes = sorted(y.unique())

In [28]:
clf= MLPClassifier(hidden_layer_sizes=(3), activation='logistic' , solver='lbfgs', random_state = 1)

In [29]:
clf.fit(x,y)
clf.predict(x)

/Users/nada/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


array([0, 0, 0, ..., 0, 0, 0])

In [30]:
# Network structure
print('Intercepts')
print(clf.intercepts_)
print('weight')
print(clf.coefs_)

Intercepts
[array([-0.57356675,  3.49542933, -3.64484774]), array([4.81840851])]
weight
[array([[-5.36970065e-01, -2.29308206e+00, -8.25979792e+00],
       [-4.94257293e-01,  1.18774624e-01, -4.44255449e+00],
       [-2.72388902e-01, -2.61411353e-01,  4.85190407e+00],
       [-3.81270820e-02, -2.87554963e-01, -1.11111787e+00],
       [-1.83302252e+00, -2.08625632e-03, -4.73777007e+00],
       [-2.74560497e+00,  1.54971117e-02,  1.82269387e+00]]), array([[ -0.12351709],
       [-26.53231112],
       [  1.32080442]])]


In [31]:
# Prediction
print(
 pd.concat([
 taxi_df,
 pd.DataFrame(clf.predict_proba(x), columns= classes)] , axis=1)
)


        row#  user_id  vehicle_model_id  package_id  travel_type_id  \
0        1.0  17712.0              12.0         0.0             2.0   
1        2.0  17037.0              12.0         0.0             2.0   
2        3.0    761.0              12.0         0.0             2.0   
3        4.0    868.0              12.0         0.0             2.0   
4        5.0  21716.0              28.0         0.0             2.0   
...      ...      ...               ...         ...             ...   
9992  9993.0   1495.0              12.0         0.0             2.0   
9993  9994.0  21491.0              85.0         0.0             2.0   
9994  9995.0  40107.0              12.0         0.0             2.0   
9997  9998.0  24007.0              12.0         0.0             2.0   
9998  9999.0  33882.0              12.0         0.0             2.0   

      from_area_id  to_area_id  from_city_id  to_city_id       from_date  ...  \
0           1021.0      1323.0           0.0         0.0    1/1/13

In [32]:
classificationSummary(y, clf.predict(x), class_names=classes)

Confusion Matrix (Accuracy 0.9185)

       Prediction
Actual    0    1
     0 7278    0
     1  646    0


In [33]:
l = [0,0]
NN_table = pd.concat([pd.crosstab(y, clf.predict(x)) , pd.DataFrame(l)], axis=1)

<h5> 5. What’s the best technique would you choose for this problem and why?

The answer is based on the below evaluation measures.

In [34]:

def model_eval(matrix, model_name):
    tn = matrix.iloc[0,0] 
    tp = matrix.iloc[1,1] 
    fn = matrix.iloc[1,0] 
    fp = matrix.iloc[0,1] 
    tap = fn+tp
    tan = tn+fp
    tpn = tn+fn
    tpp = fp+tp
    precision = tp / tpp
    recall = tp / tap
    total = tn+tp+fn+fp
    
    
    data = [
        round((tp+tn)/total,4),
        round(1-((tp+tn)/total),4),
        round(tp /tap,4),
        round(tn /tan,4),
        round(precision,4),
                                    ]
    
    return(pd.DataFrame(data, columns=[model_name], 
                        index = ['Accuracy','Error Rate','Sensitivity','Specificity','Precision']))


In [35]:
DT = model_eval(DT_table , model_name = 'DT')
RF = model_eval(RF_table , model_name = 'RF')
NN = model_eval(NN_table , model_name = 'NN')

pd.concat([DT, RF, NN], axis=1)

<ipython-input-34-2db40e250b34>:10: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / tpp


,DT,RF,NN
Accuracy,0.8662,0.8893,0.9185
Error Rate,0.1338,0.1107,0.0815
Sensitivity,0.2421,0.1627,0.0000
Specificity,0.9202,0.9520,1.0000
Precision,0.2075,0.2265,NaN


The answer is based on the below evaluation measures.

<h6> The best technique is Random Forest (RF) becasue it yielded to higher accuracy although Decison Trees (DT) show higher ability to detect class '1' records. According to Sharma (2020), "the random forest can generalize over the data in a better way." Neural network (NN), on the other hand, was not able to detect '1' at all. 
    In general, it might be suggested to do oversampling for the rare class. 
    
    Reference: https://www.analyticsvidhya.com/blog/2020/05/decision-tree-vs-random-forest-algorithm/#:~:text=Therefore%2C%20the%20random%20forest%20can,accurate%20than%20a%20decision%20tree 
    